In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

# DataFrames

In [ ]:
import Pkg
Pkg.add(["DataFrames", "RDatasets", "Statistics", "Chain"])

In [ ]:
using DataFrames
using RDatasets # dataset
using Statistics # mean
using Chain # chain

Chargeons un jeu de données, par exemple un grand classique (raccourci pour la lisibilité du calepin) :

In [ ]:
iris = dataset("datasets", "iris")[[1, 51, 101], :]

## Sélection de colonnes

Sélectionner une ou plusieurs colonnes selon leur nom : 

In [ ]:
iris[:, :SepalLength]' # Un vecteur, pas un DataFrame !

In [ ]:
select(iris, :SepalLength, 2, "PetalLength") # Un DataFrame (fonction select)
# Par numéro de colonne aussi !
# Soit un symbole, comme :SepalLength, soit une chaîne de caractères, comme "PetalLength"

In [ ]:
select(iris, Not(2)) # Toutes les colonnes, sauf.
# Not n'est pas une fonction Julia, uniquement de DataFrames.jl.

In [ ]:
select(iris, Cols(endswith("Length"))) # Sélection selon un prédicat.
# L'appel de fonction endswith("Length") retourne une fonction : 
#     endswith("Length")("SepalLength") == true
#     endswith("Length")("PetalWidth") == false

In [ ]:
select(iris, :Species, Not(:Species)) # D'abord la colonne Species, puis les autres.
# On peut aussi faire (cas particulier de la première colonne) : select(iris, :Species, :)
# : pour toutes les colonnes, les doublons ne sont pas autorisés

## Transformation des données

### Agrégation sur une colonne complète

`combine` fusionne toutes les lignes en une seule avec l'opération indiquée.

Pour appliquer une fonction comme `mean` sur toute une colonne :

In [ ]:
combine(iris, :PetalLength => mean)

Pour récupérer la valeur seule, on peut utiliser le DataFrame comme une matrice : 

In [ ]:
combine(iris, :PetalLength => mean)[1, 1]

**Avec `select`, on effectue l'opération ligne par ligne**, donc pour chaque ligne une moyenne d'une seule valeur :

### Création de nouvelles colonnes

`select` permet de créer une nouvelle colonne avec un calcul simple, donné par une fonction à appliquer sur toute la colonne. Par exemple, pour calculer la moyenne des longueurs de pétales sur tout le DataFrame et recopier cette valeur moyenne dans chaque ligne :

In [ ]:
select(iris, :PetalLength => mean)

On peut transformer plusieurs colonnes à la fois et générer une colonne, la fonction précisée doit avoir le bon nombre d'arguments :

In [ ]:
select(iris, [:PetalLength, :PetalWidth] => +)

Sinon, on peut définir une fonction anonyme qui prendra en argument les colonnes complètes (`.+` effectue l'opération `+` ligne par ligne) :

In [ ]:
select(iris, [:PetalLength, :PetalWidth] => (pl, pw) -> (pl .+ pw) ./ (pl .- pw))

On peut aussi appliquer plusieurs fonctions sur plusieurs colonnes, ce qui produit le résultat de chaque fonction pour chaque colonne (produit cartésien) avec `.=>` au lieu de `=>` :

In [ ]:
select(iris, [:PetalLength, :PetalWidth] .=> [minimum, maximum])

La fonction pour la transformation est censée prendre toute la colonne en entrée, pas juste la valeur pour une ligne. Sinon, on peut utiliser `ByRow` :

In [ ]:
select(iris, :PetalLength => sqrt)

In [ ]:
select(iris, :PetalLength => ByRow(sqrt))

On peut aussi produire plusieurs valeurs à partir d'une ou plusieurs colonnes en renvoyant un tuple, puis en utilisant une transformation `AsTable` :

In [ ]:
select(iris, :PetalLength => ByRow((pl) -> (pl, pl.^2)) => AsTable)

`transform` a la même syntaxe que `select`, mais ajoute des colonnes (au lieu de ne retourner que les colonnes mentionnées) : 

In [ ]:
transform(iris, :PetalLength => ByRow((pl) -> (pl, pl.^2)) => AsTable)

Pour donner des noms, on peut utiliser un tuple nommé :

In [ ]:
transform(iris, :PetalLength => ByRow((pl) -> (pl=pl, pl_sq=pl.^2)) => AsTable)

> `combine` : fusionner toutes les lignes (ou le groupe).
> 
> `select` : opération ligne par ligne ; le DataFrame retourné n'a que les colonnes mentionnées. 
> 
> `transform` : opération ligne par ligne ; on ajoute des colonnes au DataFrame.

## Groupements de lignes

In [ ]:
# Les résultats seront plus intéressants avec toutes les données.
iris = dataset("datasets", "iris");

`groupby` sert à grouper des lignes selon un critère, on peut alors appliquer `combine` pour produire une valeur par groupe, comme une moyenne :

In [ ]:
combine(
    groupby(iris, :Species),
    Not(:Species) .=> mean)

Pour des suites d'opérations, on peut aussi utiliser une syntaxe alternative, avec une transformation par ligne, sans préciser chaque fois l'objet DataFrame sur lequel l'opération se déroule :

In [ ]:
@chain iris begin
    groupby(:Species)
    combine(Not(:Species) .=> mean)
end